# 0.0. IMPORTS

## 0.1. Libraries

In [53]:
import pandas as pd
import numpy as np

from IPython.display import Image

## 0.2. Functions

### 0.2.1. Get Attributes

In [2]:
def get_numerical_attributes(data):
    return data.select_dtypes(include=['int64', 'float64'])

In [3]:
def get_categorical_attributes(data):
    return data.select_dtypes(exclude=['int64', 'flaot64', 'datetime64[ns]'])

### 0.2.2. Get Descriptive Info

In [4]:
def get_descriptive_info(data):
    num_attributes = get_numerical_attributes(data)
    
    # Central Tendency - mean, median
    df_mean = pd.DataFrame(num_attributes.apply(np.mean)).T
    df_median = pd.DataFrame(num_attributes.apply(np.median)).T
    
    # Dipersion - std, min, max, range, skew, kurtosis
    df_std = pd.DataFrame(num_attributes.apply(np.std)).T
    df_min = pd.DataFrame(num_attributes.apply(min)).T
    df_max = pd.DataFrame(num_attributes.apply(max)).T
    df_range = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
    df_skew = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    df_kurtosis = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T
    
    # min, max, range, median, std, skew, kurtosis
    df_descriptive_info = pd.concat([df_min, df_max, df_range,
                                     df_mean, df_median, df_std,
                                     df_skew, df_kurtosis]).T.reset_index()
    
    df_descriptive_info.columns = ['attributes', 'min', 'max',
                                   'range', 'mean', 'median',
                                   'std', 'skew', 'kurtosis']
    
    return df_descriptive_info

### 0.2.3. Analyze Outliers

In [5]:
def find_and_analyze_outliers(data):
    num_attributes = get_numerical_attributes(data)
    
    Q1 = num_attributes.quantile(0.25)
    Q3 = num_attributes.quantile(0.75)
    IQR = Q3 - Q1
    
    inf_lim = Q1 - 1.5 * IQR
    sup_lim = Q3 + 1.5 * IQR
    
    outliers = ((num_attributes < inf_lim) | (num_attributes > sup_lim))
    
    outliers_count = outliers.sum()
    outliers_percentage = (outliers_count / len(num_attributes)) * 100
    
    df_outliers = data[outliers.any(axis=1)]
    
    df_outliers_analyze = pd.DataFrame({
        'Outliers Count' : outliers_count,
        'Outliers Percentage' : outliers_percentage
    })
    
    df_outliers_analyze = df_outliers_analyze[df_outliers_analyze['Outliers Count'] > 0]
    
    df_outliers_analyze = df_outliers_analyze.sort_values(by='Outliers Count', ascending = False)
    
    return df_outliers, df_outliers_analyze

## 0.3. Loading Data

In [6]:
df_raw = pd.read_csv("../data/raw/teste_indicium_precificacao.csv")

# 1.0. DESCRIPTION DATA

In [8]:
df1 = df_raw.copy()

In [9]:
df1.sample(5).T

,983,26395,3843,2452,33279
id,396636,21016290,2325144,1228561,26278431
nome,Stylish Designer Studio with Piano,Large 1bdr family friendly apt close to midtown,Cozy Nook in a Unique Loft,Stylish Apt in Heart of Ft. Greene,HUGE Private Bedroom in Midtown Manhattan!
host_id,1981742,14085769,11876825,2305477,42624482
host_name,Daniel,Pj,Christopher,Amy,Param
bairro_group,Manhattan,Queens,Brooklyn,Brooklyn,Manhattan
bairro,Harlem,Maspeth,Bushwick,Fort Greene,Midtown
latitude,40.83091,40.72543,40.69856,40.68764,40.74897
longitude,-73.94223,-73.89666,-73.93162,-73.97545,-73.98642
room_type,Entire home/apt,Entire home/apt,Private room,Entire home/apt,Private room
price,135,125,128,142,120


## 1.1. Data Dimension

In [10]:
print(f'Number of Rows: {df1.shape[0]}')
print(f'Number of Features: {df1.shape[1]}')

Number of Rows: 48894
Number of Features: 16


## 1.2. Data Types

In [11]:
df1.dtypes

id                                 int64
nome                              object
host_id                            int64
host_name                         object
bairro_group                      object
bairro                            object
latitude                         float64
longitude                        float64
room_type                         object
price                              int64
minimo_noites                      int64
numero_de_reviews                  int64
ultima_review                     object
reviews_por_mes                  float64
calculado_host_listings_count      int64
disponibilidade_365                int64
dtype: object

## 1.3. Check Duplicated

In [12]:
df1.duplicated().sum()

0

## 1.4. Check NA

In [13]:
df1.isna().sum()

id                                   0
nome                                16
host_id                              0
host_name                           21
bairro_group                         0
bairro                               0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimo_noites                        0
numero_de_reviews                    0
ultima_review                    10052
reviews_por_mes                  10052
calculado_host_listings_count        0
disponibilidade_365                  0
dtype: int64

### 1.4.1. nome

Cada row nulo receberá, respectivamente, a junção dos dados em host_name + bairro_group + bairro no seu campo.

In [14]:
aux1 = df1[df1['nome'].isna()]

aux1[['host_name', 'bairro_group', 'bairro']].groupby('host_name').count()

,bairro_group,bairro
host_name,,
Andrea,1,1
Anna,1,1
Carolina,1,1
Huei-Yin,1,1
Jeff,1,1
Jesse,1,1
Jonathan,1,1
Josh,1,1
Juliette,1,1


### 1.4.2. host_name
Cada row nulo receberá, respectivamente, o dado do row nome.

In [15]:
aux2 = df1[df1['host_name'].isna()]

aux2['nome'].duplicated().any()

False

### 1.4.3. ultima_review e reviews_por_mes
Ambas as colunas tem respectivamente 10.052 dados nulos, nas mesmas posições, o que **equivale a aproximadamente 20.56%** do total dos dados. Isso sem considerar a destribuição os dados dentro desses 20.56%, onde por exemplo, pode haver 90% dos dados do Bronx.

In [16]:
# Onde é nulo em ultima_review também é em reviews_por_mes
df1['ultima_review'].isna().equals(df1['reviews_por_mes'].isna())

True

In [17]:
# A quantidade de dados onde o numero_de_reviews é igual a 0 é a mesma referente a dados nulos em ultima_review e reviews_por_mes
df1[df1['numero_de_reviews'] == 0].shape

(10052, 16)

In [18]:
# Onde os dados de ultima_review e reviews_por_mes são nulos, os dados são igual a 0. Confirmando que nunca houve aluguel desses imoveis.
aux= df1[df1['ultima_review'].isna()]
aux['numero_de_reviews'].unique()

array([0], dtype=int64)

In [19]:
# Data mais antiga dos dados
pd.to_datetime(df1['ultima_review']).min()

Timestamp('2011-03-28 00:00:00')

In [20]:
# menor valjor atribuido a review_por_mes
df1['reviews_por_mes'].min()

0.01

## 1.5. Tratamento de nulos

In [21]:
# nome
df1['nome'] = df1.apply(lambda x: f"{x['host_name']}  {x['bairro_group']} {x['bairro']}" if pd.isna(x['nome']) else x['nome'], axis=1)

# host_name
df1['host_name'] = df1.apply(lambda x: x['nome'] if pd.isna(x['host_name']) else x['host_name'], axis=1)

# ultima_review
df1['ultima_review'] = df1.groupby('bairro_group')['ultima_review'].transform(fillna_mode)
df1['ultima_review'] = pd.to_datetime(df1['ultima_review'])

# reviews_por_mes
df1['reviews_por_mes'] = df1.apply(lambda x: -1 if pd.isna(x['reviews_por_mes']) else x['reviews_por_mes'], axis=1)

In [22]:
df1.isna().sum()

id                               0
nome                             0
host_id                          0
host_name                        0
bairro_group                     0
bairro                           0
latitude                         0
longitude                        0
room_type                        0
price                            0
minimo_noites                    0
numero_de_reviews                0
ultima_review                    0
reviews_por_mes                  0
calculado_host_listings_count    0
disponibilidade_365              0
dtype: int64

## 1.6. Data Descriptive Info

In [23]:
get_descriptive_info(df1)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,id,2595.00000,3.648724e+07,3.648465e+07,1.901753e+07,1.967743e+07,1.098277e+07,-0.090265,-1.227738
1,host_id,2438.00000,2.743213e+08,2.743189e+08,6.762139e+07,3.079553e+07,7.861037e+07,1.206189,0.169041
2,latitude,40.49979,4.091306e+01,4.132700e-01,4.072895e+01,4.072308e+01,5.452883e-02,0.237157,0.148937
3,longitude,-74.24442,-7.371299e+01,5.314300e-01,-7.395217e+01,-7.395568e+01,4.615665e-02,1.284179,5.021498
4,price,0.00000,1.000000e+04,1.000000e+04,1.527208e+02,1.060000e+02,2.401542e+02,19.118743,585.660822
5,minimo_noites,1.00000,1.250000e+03,1.249000e+03,7.030085e+00,3.000000e+00,2.051053e+01,21.827092,854.056640
6,numero_de_reviews,0.00000,6.290000e+02,6.290000e+02,2.327476e+01,5.000000e+00,4.455054e+01,3.690589,19.529325
7,reviews_por_mes,-1.00000,5.850000e+01,5.950000e+01,8.853403e-01,3.700000e-01,1.778533e+00,2.452605,28.687688
8,calculado_host_listings_count,1.00000,3.270000e+02,3.260000e+02,7.144005e+00,1.000000e+00,3.295252e+01,7.933091,67.549426
9,disponibilidade_365,0.00000,3.650000e+02,3.650000e+02,1.127762e+02,4.500000e+01,1.316173e+02,0.763459,-0.997430


## 1.7. Find and Analyze Outliers

In [24]:
df_outliers, df_outliers_analyze = find_and_analyze_outliers(df1)

In [25]:
df_outliers_analyze

,Outliers Count,Outliers Percentage
calculado_host_listings_count,7080,14.480304
minimo_noites,6607,13.512905
numero_de_reviews,6021,12.314394
reviews_por_mes,3312,6.773837
price,2972,6.078455
longitude,2832,5.792122
host_id,1526,3.121037
latitude,425,0.869227


## 1.8. Saved actual data

In [26]:
path = '..\data\processed\df1_description_data.csv'
df1.to_csv(path, index=False)

# 2.0. FEATURE ENGINEERING

In [27]:
df2 = df1.copy()

In [28]:
df2.sample(5).T

,26860,37871,42707,42212,3833
id,21298222,29965497,33155206,32775747,2314398
nome,Cozy private studio TimesSquare Perfect Location,Entire 3 FL apartment close to Subway,"Central, spacious, beautiful, well-appointed 1-br",Super cute & cozy 2BR home in Brooklyn,Duplex Brownstone sleeps 4-6
host_id,110081618,67987135,13564519,16479647,11246260
host_name,Harry,Siyi,Rebecca,Rebecca,Claudia
bairro_group,Manhattan,Brooklyn,Manhattan,Brooklyn,Brooklyn
bairro,Hell's Kitchen,Bedford-Stuyvesant,Midtown,Bedford-Stuyvesant,Cobble Hill
latitude,40.7617,40.69321,40.76309,40.68376,40.68554
longitude,-73.98931,-73.93835,-73.98063,-73.93336,-73.99499
room_type,Private room,Entire home/apt,Entire home/apt,Entire home/apt,Entire home/apt
price,150,139,190,99,175


In [29]:
# ano
df2['ano'] = df2['ultima_review'].dt.year

# mes
df2['mes'] = df2['ultima_review'].dt.month

# dia
df2['dia'] = df2['ultima_review'].dt.day

# semana do ano
df2['semana_do_ano'] = df2['ultima_review'].dt.isocalendar().week

# ano_semana
df2['ano_semana'] = df2['ultima_review'].dt.strtime('%Y-%W')

# bairro
bairro_media = df2.groupby('bairro')['price'].mean()
df2['bairro_media'] = df2['bairro'].map(bairro_media)

# bairro_group_media
bairro_group_media = df2.groupby('bairro_group')['price'].mean()
df2['bairro_group_media'] = df2['bairro_group'].map(bairro_group_media)

# encontra a semana do ano com o maior numero de aluguéis para os bairros
df2['bairro_semana_pico_aluguel'] = df2.groupby('bairro')['semana_do_ano'].transform(lambda x: x.mode().iloc[0])

# encontra a semana do ano com o maior número de aluguéis para cada bairro
df2['bairro_group_semana_pico_aluguel'] = df2.groupby('bairro_group')['semana_do_ano'].transform(lambda x: x.mode().iloc[0])

# encontra o room_type mais comum nos bairros
df2['room_type_bairro'] = df2.groupby('bairro')['room_type'].transform(lambda x: x.mode().iloc[0])

# encontra o room_type mais comum nos bairros_group
df2['room_type_bairro_group'] = df2.groupby('bairro_group')['room_type'].transform(lambda x: x.mode().iloc[0])

# encontre a quantidade minima de noite para alguel mais comum por bairro
df2['minimo_noites_bairro'] = df2.groupby('bairro')['minimo_noites'].transform(lambda x: x.mode().iloc[0])

# encontre a quantidade minima de noite para alguel mais comum por bairro_group
df2['minimo_noites_bairro_group'] = df2.groupby('bairro_group')['minimo_noites'].transform(lambda x: x.mode().iloc[0])

In [45]:
df2.sample(5).T

,34105,2313,11512,46146,31662
id,27029402,1121497,8953625,35117050,24673759
nome,Central Park Spacious Room,Spacious E Williamsburg with Yard!!,"PRIVATE, CLEAN, ROOM CLOSE TO MANH",Room for rent. WiFi included. Near shopping ce...,Williamsburg Pad
host_id,194843581,496164,45416627,84260385,25895777
host_name,Hongzhi,Todd,Lolita,Samantha,Vincent
bairro_group,Manhattan,Brooklyn,Queens,Queens,Brooklyn
bairro,Upper West Side,Williamsburg,Astoria,Corona,Williamsburg
latitude,40.79908,40.70974,40.76832,40.73505,40.70843
longitude,-73.96215,-73.93962,-73.92346,-73.86144,-73.96833
room_type,Private room,Private room,Private room,Private room,Entire home/apt
price,87,75,48,70,400


## 2.1 Saved Actual data

In [50]:
path = '..\data\processed\df2_description_data.csv'
df1.to_csv(path, index=False)